# Ejercicio Módulo 6 - Machine Learning con PySpark

## Dataset: Diamonds

### **Objetivos:**

1. **Carga de datos (10%)**
   - Cargar el dataset `diamonds.csv` desde:
     - `https://raw.githubusercontent.com/mwaskom/seaborn-data/refs/heads/master/diamonds.csv`
   - Definir un esquema explícito para los datos.

2. **Pipeline de regresión (40%)**
   - Predecir la variable `price`.
   - Aplicar preprocesamiento con:
     - `Imputer`
     - `StringIndexer`
     - `OneHotEncoder`
     - `MinMaxScaler` o `StandardScaler`
     - `VectorAssembler`
   - Utilizar un modelo de regresión (ejemplo: `RandomForestRegressor`).

3. **Pipeline de clasificación (40%)**
   - Predecir la variable `cut` (multiclase).
   - Aplicar preprocesamiento similar al de la regresión.
   - Utilizar un modelo de clasificación (ejemplo: `MultiLayerPerceptronClassifier`).

4. **GridSearch con CrossValidation (10%)**
   - Aplicar `CrossValidator` con `GridSearch` para optimizar hiperparámetros en uno de los pipelines.


# Importaciones

In [71]:
# PySpark SQL
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, when, round
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Descarga de datos
import requests

# PySpark ML - Preprocesamiento
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder, MinMaxScaler, VectorAssembler

# PySpark ML - Modelos
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, OneVsRest, LogisticRegression, MultilayerPerceptronClassifier

# PySpark ML - Evaluación y optimización
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# 1. Carga de Datos (10%)

En esta sección se carga el dataset **"diamonds.csv"** desde la siguiente fuente:  
🔗 [Dataset en GitHub](https://raw.githubusercontent.com/mwaskom/seaborn-data/refs/heads/master/diamonds.csv)  

### Pasos:
1. **Descargar los datos** desde la URL y guardarlos en un archivo local.
2. **Definir un esquema explícito** con los tipos de datos adecuados.
3. **Cargar los datos en un DataFrame de PySpark** utilizando el esquema definido.
4. **Mostrar los primeros registros y la estructura del DataFrame** para verificar la correcta importación.

---

In [3]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("DiamondsML").getOrCreate()

# Descargar dataset
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/refs/heads/master/diamonds.csv"
csv_path = "diamonds.csv"

with open(csv_path, 'wb') as file:
    file.write(requests.get(url).content)

# Definir esquema del dataset
schema = StructType([
    StructField("carat", FloatType(), True),
    StructField("cut", StringType(), True),
    StructField("color", StringType(), True),
    StructField("clarity", StringType(), True),
    StructField("depth", FloatType(), True),
    StructField("table", FloatType(), True),
    StructField("price", IntegerType(), True),
    StructField("x", FloatType(), True),
    StructField("y", FloatType(), True),
    StructField("z", FloatType(), True)
])

# Cargar datos con esquema
df = spark.read.csv(csv_path, header=True, inferSchema=False, schema=schema)

# Mostrar primeros registros y esquema
df.show(5)
df.printSchema()


+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
| 0.21|Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
| 0.23|   Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
| 0.29|Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
| 0.31|   Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
only showing top 5 rows

root
 |-- carat: float (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: float (nullable = true)
 |-- table: float (nullable = true)
 |-- price: integer (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)



# 2. Pipeline de Regresión (40%)  

En esta sección se construye un **Pipeline de Regresión** para predecir el precio (`price`) de los diamantes utilizando modelos de Machine Learning en **PySpark MLlib**.  

### Pasos:
1. **Preprocesamiento de Datos**  
   - Manejo de valores nulos con `Imputer`.  
   - Codificación de variables categóricas con `StringIndexer` y `OneHotEncoder`.  
   - Escalado de variables numéricas con `MinMaxScaler`.  
   - Ensamblaje de todas las características en una sola columna (`features`).  

2. **Modelado**  
   - Se prueban dos modelos de regresión:  
     - `RandomForestRegressor`   
     - `GBTRegressor`   
   - Se entrena cada modelo con los datos de entrenamiento.  

3. **Evaluación de Modelos**  
   - Se calculan métricas de rendimiento:  
     - **R²** (coeficiente de determinación)  
     - **RMSE** (error cuadrático medio raíz)  
     - **MAE** (error absoluto medio)  
     - **MSE** (error cuadrático medio)  
   - Se compara el desempeño de ambos modelos y se selecciona el mejor.  

4. **Guardado del Mejor Modelo**  
   - Se almacena el modelo con mejor rendimiento para futuras predicciones.  

---

In [4]:
# Contar valores nulos
df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).show()

+-----+---+-----+-------+-----+-----+-----+---+---+---+
|carat|cut|color|clarity|depth|table|price|  x|  y|  z|
+-----+---+-----+-------+-----+-----+-----+---+---+---+
|    0|  0|    0|      0|    0|    0|    0|  0|  0|  0|
+-----+---+-----+-------+-----+-----+-----+---+---+---+



In [5]:
from pyspark.sql.types import NumericType

# Identificar columnas numéricas y categóricas
numerical_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, NumericType) and field.name != 'price']
categorical_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
label_col = 'price'

print(numerical_cols)
print(categorical_cols)

# Crear nueva columna "label" sin modificar "price"
df = df.withColumn("label", col("price"))

df.show(5)

['carat', 'depth', 'table', 'x', 'y', 'z']
['cut', 'color', 'clarity']
+-----+-------+-----+-------+-----+-----+-----+----+----+----+-----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|label|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+-----+
| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|  326|
| 0.21|Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|  326|
| 0.23|   Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|  327|
| 0.29|Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|  334|
| 0.31|   Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|  335|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+-----+
only showing top 5 rows



In [6]:
# Indexar columnas categóricas sin sobrescribir las originales
indexers_features = [
    StringIndexer(inputCol=c, outputCol=c + "_indexed", handleInvalid="keep") for c in categorical_cols
]

categorical_cols_indexed = [c + "_indexed" for c in categorical_cols]

print(categorical_cols_indexed)


['cut_indexed', 'color_indexed', 'clarity_indexed']


In [7]:
# Imputar valores nulos en categóricas indexadas con la moda
imputer_categorical = Imputer(
    inputCols=categorical_cols_indexed,
    outputCols=[c + "_imputed" for c in categorical_cols_indexed],
    strategy="mode"
)
categorical_cols_imputed = [c + "_imputed" for c in categorical_cols_indexed]

print(categorical_cols_imputed)

['cut_indexed_imputed', 'color_indexed_imputed', 'clarity_indexed_imputed']


In [8]:
# One-Hot Encoding para las categóricas imputadas
encoders_onehot = [
    OneHotEncoder(inputCol=c, outputCol=c + "_onehot") for c in categorical_cols_imputed
]
categorical_cols_onehot = [c + "_onehot" for c in categorical_cols_imputed]

print(categorical_cols_onehot)

['cut_indexed_imputed_onehot', 'color_indexed_imputed_onehot', 'clarity_indexed_imputed_onehot']


In [9]:
# Imputar valores nulos en numéricas con la mediana
imputer_numerical = Imputer(
    inputCols=numerical_cols,
    outputCols=[c + "_imputed" for c in numerical_cols],
    strategy="median"
)
numerical_cols_imputed = [c + "_imputed" for c in numerical_cols]

print(numerical_cols_imputed)

['carat_imputed', 'depth_imputed', 'table_imputed', 'x_imputed', 'y_imputed', 'z_imputed']


In [10]:
# Escalar numéricas con MinMaxScaler
assembler_numerical = VectorAssembler(
    inputCols=numerical_cols_imputed,
    outputCol="numeric_features"
)
scaler = MinMaxScaler(
    inputCol="numeric_features",
    outputCol="numeric_features_scaled"
)

In [11]:
# Ensamblar todas las características
all_features = ["numeric_features_scaled"] + categorical_cols_onehot
assembler_all = VectorAssembler(inputCols=all_features, outputCol="features")

In [12]:
# Modelo de regresión
regressor = RandomForestRegressor(featuresCol="features", labelCol="price", seed=42)

In [13]:
gbt_regressor = GBTRegressor(featuresCol="features", labelCol="price", seed=42)

In [14]:
# Crear pipeline con TODAS las etapas en orden
pipeline_rf = Pipeline(stages=[
    *indexers_features,   
    imputer_categorical,  
    *encoders_onehot,     
    imputer_numerical,    
    assembler_numerical,  
    scaler,               
    assembler_all,        
    regressor   # Modelo RF
])

pipeline_gbt = Pipeline(stages=[
    *indexers_features,   
    imputer_categorical,  
    *encoders_onehot,     
    imputer_numerical,    
    assembler_numerical,  
    scaler,               
    assembler_all,        
    gbt_regressor   # Modelo GBT
])

In [15]:
# Dividir en entrenamiento (80%) y prueba (20%)
df_train, df_test = df.randomSplit([0.8, 0.2], seed=42)

# Entrenar el modelo
pipeline_model_rf = pipeline_rf.fit(df_train)
pipeline_model_gbt = pipeline_gbt.fit(df_train)

# Hacer predicciones en el conjunto de prueba
df_pred_rf = pipeline_model_rf.transform(df_test)
df_pred_gbt = pipeline_model_gbt.transform(df_test)

In [16]:
# Evaluadores para las métricas de regresión
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")

# Calcular métricas
r2_rf = evaluator_r2.evaluate(df_pred_rf)
r2_gbt = evaluator_r2.evaluate(df_pred_gbt)
rmse_rf = evaluator_rmse.evaluate(df_pred_rf)
rmse_gbt = evaluator_rmse.evaluate(df_pred_gbt)
mae_rf = evaluator_mae.evaluate(df_pred_rf)
mae_gbt = evaluator_mae.evaluate(df_pred_gbt)
mse_rf = evaluator_mse.evaluate(df_pred_rf)
mse_gbt = evaluator_mse.evaluate(df_pred_gbt)


print("\n**Comparación de Modelos de Regresión**")
print(f"RandomForestRegressor - R²: {r2_rf:.4f} | RMSE: {rmse_rf:.2f} | MAE: {mae_rf:.2f} | MSE: {mse_rf:.2f}")
print(f"GBTRegressor - R²: {r2_gbt:.4f} | RMSE: {rmse_gbt:.2f} | MAE: {mae_gbt:.2f} | MSE: {mse_gbt:.2f}")

# Seleccionar el mejor modelo
best_pipeline = pipeline_model_gbt if r2_gbt > r2_rf else pipeline_model_rf
print(f"\nMejor modelo seleccionado: {'GBTRegressor' if r2_gbt > r2_rf else 'RandomForestRegressor'}")

# Guardar el mejor modelo
best_pipeline.write().overwrite().save("models/best_diamond_regression")


**Comparación de Modelos de Regresión**
RandomForestRegressor - R²: 0.9070 | RMSE: 1229.87 | MAE: 684.11 | MSE: 1512592.39
GBTRegressor - R²: 0.9462 | RMSE: 935.20 | MAE: 501.64 | MSE: 874589.89

Mejor modelo seleccionado: GBTRegressor


# 4. GridSearch con CrossValidation (10%)  

En esta sección se optimiza el **modelo de regresión** utilizando **Validación Cruzada** y **Búsqueda en Cuadrícula (GridSearch)**.  

### Pasos:
1. **Definir hiperparámetros a ajustar**  
   - `numTrees`: número de árboles en el bosque aleatorio (`[10, 20, 30]`).  
   - `maxDepth`: profundidad máxima de cada árbol (`[5, 10, 15]`).  

2. **Aplicar Validación Cruzada**  
   - Se usa **3-Fold Cross Validation**.  
   - Se mide el rendimiento con **R²**.  
   - Se entrena el **GBTRegressor** con todas las combinaciones de hiperparámetros.  

3. **Evaluación del Mejor Modelo**  
   - Se selecciona la mejor combinación de hiperparámetros.  
   - Se calculan las métricas:  
     - **R²**  
     - **RMSE**  
     - **MAE**  
     - **MSE**  

4. **Guardar el Mejor Modelo**  
   - Se almacena el modelo optimizado en `models/diamond_regression_best`.  

---

In [17]:
# Definir la cuadrícula de hiperparámetros
paramGrid_regression = (
    ParamGridBuilder()
    .addGrid(regressor.numTrees, [10, 20, 30])  # Número de árboles
    .addGrid(regressor.maxDepth, [5, 10, 15])  # Profundidad máxima
    .build()
)

# Configurar CrossValidator
crossval_regression = CrossValidator(
    estimator=pipeline_gbt,  # Usamos el pipeline de regresión
    estimatorParamMaps=paramGrid_regression,  # Hiperparámetros
    evaluator=evaluator_r2,  # Evaluamos con R²
    numFolds=3,  # 3-Fold Cross Validation
    parallelism=4,  # Procesamiento paralelo
    seed=42
)

# Entrenar el modelo optimizado
cv_model_regression = crossval_regression.fit(df_train)

# Hacer predicciones con el mejor modelo
df_pred_cv_regression = cv_model_regression.transform(df_test)

# Evaluar el mejor modelo
r2_cv = evaluator_r2.evaluate(df_pred_cv_regression)
rmse_cv = evaluator_rmse.evaluate(df_pred_cv_regression)
mae_cv = evaluator_mae.evaluate(df_pred_cv_regression)
mse_cv = evaluator_mse.evaluate(df_pred_cv_regression)

# Mostrar resultados
print("\n**Evaluación del Mejor Modelo de Regresión (GridSearch + CrossValidation)**")
print(f"R²: {r2_cv:.4f}  (Cuanto más cercano a 1, mejor)")
print(f"RMSE: {rmse_cv:.2f}  (Error cuadrático medio raíz, cuanto menor mejor)")
print(f"MAE: {mae_cv:.2f}  (Error absoluto medio, cuanto menor mejor)")
print(f"MSE: {mse_cv:.2f}  (Error cuadrático medio, cuanto menor mejor)")

# Guardar el modelo optimizado
cv_model_regression.bestModel.write().overwrite().save("models/diamond_regression_best")

print("\nMejor modelo de regresión guardado con éxito.")


**Evaluación del Mejor Modelo de Regresión (GridSearch + CrossValidation)**
R²: 0.9462  (Cuanto más cercano a 1, mejor)
RMSE: 935.20  (Error cuadrático medio raíz, cuanto menor mejor)
MAE: 501.64  (Error absoluto medio, cuanto menor mejor)
MSE: 874589.89  (Error cuadrático medio, cuanto menor mejor)

Mejor modelo de regresión guardado con éxito.


In [18]:
# Cargar el mejor modelo de regresión
best_regression_model = PipelineModel.load("models/diamond_regression_best")

# Hacer predicciones en el conjunto de prueba
df_pred_best_regression = best_regression_model.transform(df_test)

# Evaluar el modelo cargado
r2_best = evaluator_r2.evaluate(df_pred_best_regression)
rmse_best = evaluator_rmse.evaluate(df_pred_best_regression)
mae_best = evaluator_mae.evaluate(df_pred_best_regression)
mse_best = evaluator_mse.evaluate(df_pred_best_regression)

# Mostrar resultados
print("\n**Evaluación del Mejor Modelo de Regresión Cargado**")
print(f"R²: {r2_best:.4f}  (Cuanto más cercano a 1, mejor)")
print(f"RMSE: {rmse_best:.2f}  (Error cuadrático medio raíz, cuanto menor mejor)")
print(f"MAE: {mae_best:.2f}  (Error absoluto medio, cuanto menor mejor)")
print(f"MSE: {mse_best:.2f}  (Error cuadrático medio, cuanto menor mejor)")

# Mostrar algunas predicciones
df_pred_best_regression.select("features", "label", "prediction").show(10)


**Evaluación del Mejor Modelo de Regresión Cargado**
R²: 0.9462  (Cuanto más cercano a 1, mejor)
RMSE: 935.20  (Error cuadrático medio raíz, cuanto menor mejor)
MAE: 501.64  (Error absoluto medio, cuanto menor mejor)
MSE: 874589.89  (Error cuadrático medio, cuanto menor mejor)
+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|(23,[1,2,3,4,5,6,...|  367| 559.1611988232905|
|(23,[1,2,3,4,5,7,...|  367|498.98727992519423|
|(23,[1,2,3,4,5,7,...|  367| 509.6352558537926|
|(23,[0,1,2,3,4,5,...|  386| 617.6673701952654|
|(23,[0,1,2,3,4,5,...|  386| 519.3629843189983|
|(23,[0,1,2,3,4,5,...|  404| 519.3629843189983|
|(23,[0,1,2,3,4,5,...|  452| 733.5297114669011|
|(23,[0,1,2,3,4,5,...|  439| 650.9265566085334|
|(23,[0,1,2,3,4,5,...|  376| 643.3707221153938|
|(23,[0,1,2,3,4,5,...|  442|509.21243595786325|
+--------------------+-----+------------------+
only showing top 10 rows



In [19]:
# Seleccionar las columnas de interés y redondear la predicción para mejor visualización
df_comparison_regression = df_pred_best_regression.select(
    col("label").alias("Real Price"), 
    round(col("prediction"), 2).alias("Predicted Price")
)

# Mostrar los primeros 15 valores reales vs. predicciones
df_comparison_regression.show(15)

+----------+---------------+
|Real Price|Predicted Price|
+----------+---------------+
|       367|         559.16|
|       367|         498.99|
|       367|         509.64|
|       386|         617.67|
|       386|         519.36|
|       404|         519.36|
|       452|         733.53|
|       439|         650.93|
|       376|         643.37|
|       442|         509.21|
|       357|          494.9|
|       458|         728.42|
|       462|         617.18|
|       395|         531.13|
|       548|         559.16|
+----------+---------------+
only showing top 15 rows



# 3. Pipeline de Clasificación (40%)  

En esta sección se entrena un modelo de **clasificación multiclase** para predecir la variable `cut` del diamante.  

## Pasos:  

### **1 Preprocesamiento de Datos**  
- **Categóricas**:  
  - Se convierten a valores numéricos con `StringIndexer`.  
  - Se imputan valores nulos con `Imputer`.  
  - Se aplica `OneHotEncoder`.  
- **Numéricas**:  
  - Se imputan valores nulos con `Imputer`.  
  - Se normalizan con `MinMaxScaler`.  
- **VectorAssembler** para combinar todas las características en una única columna `features`.  

### **2️ Modelos de Clasificación**  
- **RandomForestClassifier**  
- **OneVsRest (Logistic Regression)**  

### **3️ Creación de Pipelines**  
- Se crean **dos pipelines de clasificación**, uno con **RandomForest** y otro con **OneVsRest**.  

### **4️ Entrenamiento y Evaluación**  
- Se entrenan ambos modelos en los datos de entrenamiento.  
- Se hacen predicciones en el conjunto de prueba.  
- Se calculan las métricas de evaluación:  
  - **Accuracy**  
  - **F1-Score**  
  - **Precision**  
  - **Recall**  

### **5️ Selección del Mejor Modelo**  
- Se compara el rendimiento de ambos modelos.  
- Se guarda el modelo con **mejor rendimiento** en `models/best_diamond_classification`.  

---

In [20]:
# Crear una copia del DataFrame para clasificación
df_classification = df.select("*")

In [21]:
# Eliminar la columna "label" si ya existe en df_classification
if "label" in df_classification.columns:
    df_classification = df_classification.drop("label")

In [22]:
# Variables categóricas (sin incluir la label 'cut')
categorical_cols_classification = [col for col in categorical_cols if col != "cut"]

# Variables numéricas (agregando 'price')
numerical_cols_classification = numerical_cols + ["price"]

# Definir la columna a predecir
label_col_classification = "cut"

print("Categóricas:", categorical_cols_classification)
print("Numéricas:", numerical_cols_classification)

Categóricas: ['color', 'clarity']
Numéricas: ['carat', 'depth', 'table', 'x', 'y', 'z', 'price']


In [23]:
# Indexar la variable objetivo (cut) convirtiéndola en label
indexer_label = StringIndexer(inputCol="cut", outputCol="label", handleInvalid="keep")

In [24]:
# Indexar las features categóricas
indexers_features_classification = [
    StringIndexer(inputCol=c, outputCol=c + "_indexed", handleInvalid="keep") 
    for c in categorical_cols_classification
]

# Lista de columnas indexadas
categorical_cols_indexed_classification = [c + "_indexed" for c in categorical_cols_classification]

print("Columnas categóricas indexadas:", categorical_cols_indexed_classification)

Columnas categóricas indexadas: ['color_indexed', 'clarity_indexed']


In [25]:
# Imputar con la moda las columnas categóricas indexadas
imputer_categorical_classification = Imputer(
    inputCols=categorical_cols_indexed_classification,
    outputCols=[c + "_imputed" for c in categorical_cols_indexed_classification],
    strategy="mode"
)

# Lista de columnas categóricas indexadas e imputadas
categorical_cols_imputed_classification = [c + "_imputed" for c in categorical_cols_indexed_classification]

print("Columnas categóricas imputadas:", categorical_cols_imputed_classification)

Columnas categóricas imputadas: ['color_indexed_imputed', 'clarity_indexed_imputed']


In [26]:
# Aplicar One-Hot Encoding a las categóricas indexadas e imputadas
encoders_onehot_classification = [
    OneHotEncoder(inputCol=c, outputCol=c + "_onehot") 
    for c in categorical_cols_imputed_classification
]

# Lista de columnas categóricas después del One-Hot Encoding
categorical_cols_onehot_classification = [c + "_onehot" for c in categorical_cols_imputed_classification]

print("Columnas categóricas codificadas con One-Hot Encoding:", categorical_cols_onehot_classification)

Columnas categóricas codificadas con One-Hot Encoding: ['color_indexed_imputed_onehot', 'clarity_indexed_imputed_onehot']


In [27]:
# Imputar valores nulos en numéricas con la mediana
imputer_numerical_classification = Imputer(
    inputCols=numerical_cols_classification,
    outputCols=[c + "_imputed" for c in numerical_cols_classification],
    strategy="median"
)

# Lista de columnas numéricas imputadas
numerical_cols_imputed_classification = [c + "_imputed" for c in numerical_cols_classification]

print("Columnas numéricas imputadas:", numerical_cols_imputed_classification)

Columnas numéricas imputadas: ['carat_imputed', 'depth_imputed', 'table_imputed', 'x_imputed', 'y_imputed', 'z_imputed', 'price_imputed']


In [28]:

# Ensamblar las columnas numéricas imputadas en un solo vector antes de escalar
assembler_numerical_classification = VectorAssembler(
    inputCols=numerical_cols_imputed_classification,
    outputCol="numeric_features"
)

# Aplicar MinMaxScaler a las variables numéricas
scaler_classification = MinMaxScaler(
    inputCol="numeric_features",
    outputCol="numeric_features_scaled"
)

print("Normalización lista: Se escalarán las variables numéricas en un rango de 0 a 1.")

Normalización lista: Se escalarán las variables numéricas en un rango de 0 a 1.


In [29]:
# Lista de todas las características finales
all_features_classification = ["numeric_features_scaled"] + categorical_cols_onehot_classification

# Ensamblar todas las características en una única columna "features"
assembler_all_classification = VectorAssembler(
    inputCols=all_features_classification,
    outputCol="features"
)

print("Ensamblaje final: Todas las variables estarán listas para el modelo de clasificación.")

Ensamblaje final: Todas las variables estarán listas para el modelo de clasificación.


In [30]:
# Definir el modelo de clasificación
classifier = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    seed=42
)

print("Modelo de clasificación seleccionado: RandomForestClassifier")

Modelo de clasificación seleccionado: RandomForestClassifier


In [31]:
# Definir el modelo base para OneVsRest (Logistic Regression)
base_classifier = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Configurar OneVsRest
one_vs_rest_classifier = OneVsRest(classifier=base_classifier, labelCol="label", featuresCol="features")

In [32]:
# Crear el pipeline con RandomForestClassifier
pipeline_classification_rf = Pipeline(stages=[
    indexer_label,                # Indexar la variable objetivo
    *indexers_features_classification,  # Indexar categóricas
    imputer_categorical_classification,  # Imputar categóricas
    *encoders_onehot_classification,  # OneHotEncoder
    imputer_numerical_classification,  # Imputar numéricas
    assembler_numerical_classification,  # Ensamblar numéricas
    scaler_classification,  # Escalar numéricas
    assembler_all_classification,  # Ensamblar todas las features
    classifier  # Modelo de clasificación
])


# Crear el pipeline con OneVsRest
pipeline_classification_ovr = Pipeline(stages=[
    indexer_label,                
    *indexers_features_classification,  
    imputer_categorical_classification,  
    *encoders_onehot_classification,  
    imputer_numerical_classification,  
    assembler_numerical_classification,  
    scaler_classification,  
    assembler_all_classification,  
    one_vs_rest_classifier  
])

print("Pipelines de clasificación creados con éxito.")

Pipelines de clasificación creados con éxito.


In [33]:
# Particionar los datos
df_train_classification, df_test_classification = df_classification.randomSplit([0.8, 0.2], seed=42)

print(f"Datos divididos: {df_train_classification.count()} en entrenamiento, {df_test_classification.count()} en prueba.")

Datos divididos: 43083 en entrenamiento, 10857 en prueba.


In [34]:
# Entrenar el pipeline de clasificación
pipeline_model_classification_rf = pipeline_classification_rf.fit(df_train_classification)
pipeline_model_classification_ovr = pipeline_classification_ovr.fit(df_train_classification)

print("Modelos de clasificación entrenados con éxito.")

Modelos de clasificación entrenados con éxito.


In [35]:
# Realizar predicciones en los datos de prueba
df_pred_classification_rf = pipeline_model_classification_rf.transform(df_test_classification)
df_pred_classification_ovr = pipeline_model_classification_ovr.transform(df_test_classification)

# Mostrar algunas predicciones
df_pred_classification_rf.select("cut", "label", "prediction").show(10)
df_pred_classification_ovr.select("cut", "label", "prediction").show(10)

+-------+-----+----------+
|    cut|label|prediction|
+-------+-----+----------+
|  Ideal|  0.0|       0.0|
|Premium|  1.0|       2.0|
|Premium|  1.0|       2.0|
|Premium|  1.0|       2.0|
|Premium|  1.0|       2.0|
|Premium|  1.0|       2.0|
|   Good|  3.0|       2.0|
|   Good|  3.0|       2.0|
|   Good|  3.0|       2.0|
|   Good|  3.0|       0.0|
+-------+-----+----------+
only showing top 10 rows

+-------+-----+----------+
|    cut|label|prediction|
+-------+-----+----------+
|  Ideal|  0.0|       0.0|
|Premium|  1.0|       1.0|
|Premium|  1.0|       1.0|
|Premium|  1.0|       1.0|
|Premium|  1.0|       1.0|
|Premium|  1.0|       1.0|
|   Good|  3.0|       1.0|
|   Good|  3.0|       1.0|
|   Good|  3.0|       2.0|
|   Good|  3.0|       0.0|
+-------+-----+----------+
only showing top 10 rows



In [36]:
# Definir evaluadores
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Evaluar métricas
accuracy_rf = evaluator_accuracy.evaluate(df_pred_classification_rf)
accuracy_ovr = evaluator_accuracy.evaluate(df_pred_classification_ovr)
f1_rf = evaluator_f1.evaluate(df_pred_classification_rf)
f1_ovr = evaluator_f1.evaluate(df_pred_classification_ovr)
precision_rf = evaluator_precision.evaluate(df_pred_classification_rf)
precision_ovr = evaluator_precision.evaluate(df_pred_classification_ovr)
recall_rf = evaluator_recall.evaluate(df_pred_classification_rf)
recall_ovr = evaluator_recall.evaluate(df_pred_classification_ovr)

print("\n**Comparación de Modelos de Clasificación**")
print(f"RandomForestClassifier - Accuracy: {accuracy_rf:.4f} F1-Score: {f1_rf:.4f} | Precision: {precision_rf:.4f} | Recall: {recall_rf:.4f}")
print(f"GBTClassifier - Accuracy: {accuracy_ovr:.4f} F1-Score: {f1_ovr:.4f} | Precision: {precision_ovr:.4f} | Recall: {recall_ovr:.4f}")

# Seleccionar el mejor modelo
best_pipeline_classification = pipeline_model_classification_ovr if accuracy_ovr > accuracy_rf else pipeline_model_classification_rf
print(f"\nMejor modelo de clasificación: {'OneVsRest' if accuracy_ovr > accuracy_rf else 'RandomForestClassifier'}")

# Guardar el mejor modelo de clasificación
best_pipeline_classification.write().overwrite().save("models/best_diamond_classification")


**Comparación de Modelos de Clasificación**
RandomForestClassifier - Accuracy: 0.6774 F1-Score: 0.6354 | Precision: 0.6623 | Recall: 0.6774
GBTClassifier - Accuracy: 0.6161 F1-Score: 0.5639 | Precision: 0.5913 | Recall: 0.6161

Mejor modelo de clasificación: RandomForestClassifier


In [37]:
# Definir la cuadrícula de hiperparámetros para clasificación
paramGrid_classification = (
    ParamGridBuilder()
    .addGrid(classifier.numTrees, [10, 20, 30])  # Número de árboles
    .addGrid(classifier.maxDepth, [5, 10, 15])  # Profundidad máxima
    .build()
)

# Configurar CrossValidator
crossval_classification = CrossValidator(
    estimator=pipeline_classification_rf,  # Usamos el pipeline de clasificación
    estimatorParamMaps=paramGrid_classification,  # Hiperparámetros
    evaluator=evaluator_f1,  # Evaluamos con F1-Score
    numFolds=3,  # 3-Fold Cross Validation
    parallelism=4,  # Procesamiento paralelo
    seed=42
)

# Entrenar el modelo optimizado
cv_model_classification = crossval_classification.fit(df_train_classification)

# Hacer predicciones con el mejor modelo
df_pred_cv_classification = cv_model_classification.transform(df_test_classification)

# Evaluar el mejor modelo
accuracy_cv = evaluator_accuracy.evaluate(df_pred_cv_classification)
f1_cv = evaluator_f1.evaluate(df_pred_cv_classification)
precision_cv = evaluator_precision.evaluate(df_pred_cv_classification)
recall_cv = evaluator_recall.evaluate(df_pred_cv_classification)

# Mostrar resultados
print("\n**Evaluación del Mejor Modelo de Clasificación (GridSearch + CrossValidation)**")
print(f"Accuracy: {accuracy_cv:.4f} (Porcentaje de predicciones correctas)")
print(f"F1-Score: {f1_cv:.4f} (Balance entre precisión y recall)")
print(f"Precision: {precision_cv:.4f} (Exactitud de las predicciones)")
print(f"Recall: {recall_cv:.4f} (Capacidad de detectar correctamente cada clase)")

# Guardar el modelo optimizado
cv_model_classification.bestModel.write().overwrite().save("models/diamond_classification_best")

print("\nMejor modelo de clasificación guardado con éxito.")


**Evaluación del Mejor Modelo de Clasificación (GridSearch + CrossValidation)**
Accuracy: 0.7267 (Porcentaje de predicciones correctas)
F1-Score: 0.7090 (Balance entre precisión y recall)
Precision: 0.7131 (Exactitud de las predicciones)
Recall: 0.7267 (Capacidad de detectar correctamente cada clase)

Mejor modelo de clasificación guardado con éxito.


# 4. Optimización con Validación Cruzada (GridSearch + CrossValidation)  

Para mejorar el modelo de clasificación, se implementa **validación cruzada con GridSearch** para optimizar hiperparámetros.  

## Pasos:  

### **1️ Definir la cuadrícula de hiperparámetros**  
Se prueban diferentes combinaciones de los siguientes parámetros:  
- **`numTrees`** (número de árboles): `[10, 20, 30]`  
- **`maxDepth`** (profundidad del árbol): `[5, 10, 15]`  

### **2️ Configurar CrossValidator**  
- Se utiliza **validación cruzada de 3 particiones** (`numFolds=3`).  
- Se evalúa el modelo con la métrica **F1-Score**.  
- Se habilita **procesamiento paralelo** (`parallelism=4`) para mayor velocidad.  

### **3️ Entrenamiento y Selección del Mejor Modelo**  
- Se entrena el modelo optimizado.  
- Se selecciona el conjunto de hiperparámetros que maximiza el rendimiento.  

### **4️ Evaluación del Mejor Modelo**  
Se evalúan las métricas en el conjunto de prueba:  
**Accuracy** (porcentaje de predicciones correctas).  
**F1-Score** (balance entre precisión y recall).  
**Precision** (exactitud de las predicciones).  
**Recall** (capacidad de detectar correctamente cada clase).  

### **5️ Guardar el Mejor Modelo**  
El modelo optimizado se guarda en **`models/diamond_classification_best`** para futuras predicciones.  

In [38]:
# Cargar el mejor modelo de clasificación
best_classification_model = PipelineModel.load("models/diamond_classification_best")

# Hacer predicciones en el conjunto de prueba
df_pred_best_classification = best_classification_model.transform(df_test_classification)

# Evaluar el modelo cargado
accuracy_best = evaluator_accuracy.evaluate(df_pred_best_classification)
f1_best = evaluator_f1.evaluate(df_pred_best_classification)
precision_best = evaluator_precision.evaluate(df_pred_best_classification)
recall_best = evaluator_recall.evaluate(df_pred_best_classification)

# Mostrar resultados
print("\n**Evaluación del Mejor Modelo de Clasificación Cargado**")
print(f"Accuracy: {accuracy_best:.4f} (Porcentaje de predicciones correctas)")
print(f"F1-Score: {f1_best:.4f} (Balance entre precisión y recall)")
print(f"Precision: {precision_best:.4f} (Exactitud de las predicciones)")
print(f"Recall: {recall_best:.4f} (Capacidad de detectar correctamente cada clase)")

# Mostrar algunas predicciones
df_pred_best_classification.select("features", "label", "prediction").show(10)


**Evaluación del Mejor Modelo de Clasificación Cargado**
Accuracy: 0.7267 (Porcentaje de predicciones correctas)
F1-Score: 0.7090 (Balance entre precisión y recall)
Precision: 0.7131 (Exactitud de las predicciones)
Recall: 0.7267 (Capacidad de detectar correctamente cada clase)
+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(20,[1,2,3,4,5,6,...|  0.0|       0.0|
|(20,[1,2,3,4,5,6,...|  1.0|       2.0|
|(20,[1,2,3,4,5,6,...|  1.0|       1.0|
|(20,[0,1,2,3,4,5,...|  1.0|       1.0|
|(20,[0,1,2,3,4,5,...|  1.0|       2.0|
|(20,[0,1,2,3,4,5,...|  1.0|       2.0|
|(20,[0,1,2,3,4,5,...|  3.0|       2.0|
|(20,[0,1,2,3,4,5,...|  3.0|       2.0|
|(20,[0,1,2,3,4,5,...|  3.0|       2.0|
|(20,[0,1,2,3,4,5,...|  3.0|       0.0|
+--------------------+-----+----------+
only showing top 10 rows



In [39]:
# Obtener los labels originales para decodificar la predicción
labels_cut = indexer_label.fit(df_classification).labels  # Lista de nombres de los cortes

# Crear una función para mapear los índices a nombres originales
def decode_cut(index):
    return labels_cut[int(index)]

# Usar una UDF (User Defined Function) para aplicar la conversión en Spark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

decode_udf = udf(decode_cut, StringType())

# Aplicar la conversión y comparar
df_comparison_classification = df_pred_best_classification.withColumn(
    "Predicted Cut", decode_udf(col("prediction"))
).select("cut", "Predicted Cut")

# Mostrar los primeros 15 valores reales vs. predicciones
df_comparison_classification.show(15)

+-------+-------------+
|    cut|Predicted Cut|
+-------+-------------+
|  Ideal|        Ideal|
|Premium|    Very Good|
|Premium|      Premium|
|Premium|      Premium|
|Premium|    Very Good|
|Premium|    Very Good|
|   Good|    Very Good|
|   Good|    Very Good|
|   Good|    Very Good|
|   Good|        Ideal|
|   Good|         Good|
|   Good|    Very Good|
|   Good|    Very Good|
|   Good|    Very Good|
|  Ideal|        Ideal|
+-------+-------------+
only showing top 15 rows



## **Suplemento: Modelo de Red Neuronal (MultilayerPerceptronClassifier)**

Para complementar el análisis, se implementa un modelo de **Red Neuronal Artificial (MLP)** utilizando `MultilayerPerceptronClassifier` de **PySpark MLlib**.  

### **Objetivo**
Predecir la categoría de `cut` de un diamante a partir de sus características utilizando una red neuronal multicapa.  

---

### **Pasos del Modelo MLP**
#### 1️ **Carga de Datos**  
   - Se usa el mismo dataset `diamonds.csv`.  
   - Se crea una copia para el modelo MLP.  

#### 2️ **Preprocesamiento de Datos**  
   - **Indexación**: Se convierten las variables categóricas (`cut`, `color`, `clarity`) en valores numéricos con `StringIndexer`.  
   - **Imputación de valores nulos**:  
     - Categóricas: Se reemplazan con la **moda** (`Imputer` con `strategy="mode"`).  
     - Numéricas: Se reemplazan con la **mediana** (`Imputer` con `strategy="median"`).  
   - **Codificación One-Hot**: Se aplica `OneHotEncoder` a las variables categóricas indexadas (`color`, `clarity`).  
   - **Vectorización**:  
     - Se ensamblan todas las características en una columna `features` con `VectorAssembler`.  
     - Se normaliza con `MinMaxScaler` para mejorar la estabilidad del entrenamiento.  

#### 3️ **Definición de la Red Neuronal**  
   - Se determina automáticamente el **número de características** de entrada (`features`).  
   - Se establece el **número de clases** en la salida (`cut`).  
   - Se define la arquitectura de la red con **dos capas ocultas**:  
     ```python
     layers = [num_features, 64, 32, num_classes]
     ```
     Donde:  
     - `num_features`: Número total de características de entrada.  
     - `64, 32`: Capas ocultas con 64 y 32 neuronas respectivamente.  
     - `num_classes`: Número de categorías en `cut` (salida).  

#### 4️ **Entrenamiento y Evaluación**  
   - Se entrena el modelo con `MultilayerPerceptronClassifier`.  
   - Se dividen los datos en **80% entrenamiento y 20% prueba**.  
   - Se mide el rendimiento con:  
     - `Accuracy` (Exactitud).  
     - `F1-Score` (Balance entre precisión y recall).  
     - `Precision` (Exactitud de las predicciones).  
     - `Recall` (Capacidad de detectar correctamente cada clase).  

#### 5️ **Guardado del Modelo**  
   - Se almacena el modelo en `models/best_mlp_classification` para futuras predicciones.  

---

### 6 **Arquitectura del Modelo**
El modelo se construye con la siguiente arquitectura:

| Capa           | Número de Neuronas |
|---------------|--------------------|
| **Entrada**   | `num_features` (20 aprox.) |
| **Capa Oculta 1** | 64 |
| **Capa Oculta 2** | 32 |
| **Salida**    | `num_classes` (5) |

Este diseño permite capturar relaciones complejas entre las características y mejorar la predicción de la variable `cut`.  

---

### 7 **Evaluación del Modelo MLP**
Una vez entrenado, se calculan las métricas de evaluación:

- **Accuracy** (Porcentaje de predicciones correctas).  
- **F1-Score** (Balance entre precisión y recall).  
- **Precision** (Exactitud de las predicciones).  
- **Recall** (Capacidad de detectar correctamente cada clase).  

Los resultados se imprimen en la consola para su análisis.  

---

### 8 **Optimización del Modelo MLP con GridSearch y CrossValidation**

Para mejorar el rendimiento del modelo de red neuronal, se implementa **Validación Cruzada (CrossValidation)** con **GridSearch** para optimizar los hiperparámetros más relevantes.  

### **Hiperparámetros Optimizados**
Se probarán diferentes combinaciones de los siguientes parámetros:  

- **Estructura de la red (`layers`)**:
  - `[num_features, 32, num_classes]` → Red más simple.  
  - `[num_features, 64, 32, num_classes]` → Configuración actual (baseline).  
  - `[num_features, 128, 64, 32, num_classes]` → Red más profunda con más capas.  

- **Número de iteraciones (`maxIter`)**:
  - Se probarán `100` y `200` iteraciones para evaluar la convergencia del modelo.  

---

### 9 **Guardado y Uso del Modelo**
El modelo MLP entrenado se guarda en `models/best_mlp_classification`.  
Se puede recargar en el futuro para realizar nuevas predicciones sin necesidad de volver a entrenarlo.  

---

In [80]:
# Crear una copia del DataFrame original para el modelo MLP
df_mlp_classification = df_classification.select("*")

# Mostrar las primeras filas
df_mlp_classification.show(5)
df_mlp_classification.printSchema()

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
| 0.21|Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
| 0.23|   Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
| 0.29|Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
| 0.31|   Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
only showing top 5 rows

root
 |-- carat: float (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: float (nullable = true)
 |-- table: float (nullable = true)
 |-- price: integer (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)



In [81]:
# Crear el indexador para la columna "cut"
indexer_label_mlp = StringIndexer(inputCol="cut", outputCol="label", handleInvalid="keep")

# Aplicar el indexador para transformar "cut" en "label"
indexer_model = indexer_label_mlp.fit(df_mlp_classification)
df_mlp_classification = indexer_model.transform(df_mlp_classification)

# Mostrar los primeros valores de "cut" y su índice "label"
df_mlp_classification.select("cut", "label").show(10)

# Verificar las clases asignadas
labels_cut = indexer_model.labels
print("Clases indexadas:", labels_cut)

+---------+-----+
|      cut|label|
+---------+-----+
|    Ideal|  0.0|
|  Premium|  1.0|
|     Good|  3.0|
|  Premium|  1.0|
|     Good|  3.0|
|Very Good|  2.0|
|Very Good|  2.0|
|Very Good|  2.0|
|     Fair|  4.0|
|Very Good|  2.0|
+---------+-----+
only showing top 10 rows

Clases indexadas: ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']


In [82]:
# Lista de variables categóricas a indexar (sin incluir "cut" que ya está indexada)
categorical_cols_mlp = ["color", "clarity"]

# Crear indexadores para cada columna categórica
from pyspark.ml.feature import StringIndexer

indexers_features_mlp = [
    StringIndexer(inputCol=c, outputCol=c + "_indexed", handleInvalid="keep") 
    for c in categorical_cols_mlp
]

# Aplicar los indexadores uno por uno
for indexer in indexers_features_mlp:
    model = indexer.fit(df_mlp_classification)
    df_mlp_classification = model.transform(df_mlp_classification)

# Mostrar los valores transformados
df_mlp_classification.select("color", "color_indexed", "clarity", "clarity_indexed").show(10)

# Verificar que las clases han sido indexadas correctamente
print("Categorías de color indexadas:", df_mlp_classification.select("color_indexed").distinct().orderBy("color_indexed").collect())
print("Categorías de clarity indexadas:", df_mlp_classification.select("clarity_indexed").distinct().orderBy("clarity_indexed").collect())

+-----+-------------+-------+---------------+
|color|color_indexed|clarity|clarity_indexed|
+-----+-------------+-------+---------------+
|    E|          1.0|    SI2|            2.0|
|    E|          1.0|    SI1|            0.0|
|    E|          1.0|    VS1|            3.0|
|    I|          5.0|    VS2|            1.0|
|    J|          6.0|    SI2|            2.0|
|    J|          6.0|   VVS2|            4.0|
|    I|          5.0|   VVS1|            5.0|
|    H|          3.0|    SI1|            0.0|
|    E|          1.0|    VS2|            1.0|
|    H|          3.0|    VS1|            3.0|
+-----+-------------+-------+---------------+
only showing top 10 rows

Categorías de color indexadas: [Row(color_indexed=0.0), Row(color_indexed=1.0), Row(color_indexed=2.0), Row(color_indexed=3.0), Row(color_indexed=4.0), Row(color_indexed=5.0), Row(color_indexed=6.0)]
Categorías de clarity indexadas: [Row(clarity_indexed=0.0), Row(clarity_indexed=1.0), Row(clarity_indexed=2.0), Row(clarity_indexe

In [83]:
# Imputar valores nulos en categóricas indexadas
imputer_categorical_mlp = Imputer(
    inputCols=["color_indexed", "clarity_indexed"],
    outputCols=["color_imputed", "clarity_imputed"],
    strategy="mode"  # Se reemplazan nulos con el valor más frecuente
)

# Aplicar imputación
df_mlp_classification = imputer_categorical_mlp.fit(df_mlp_classification).transform(df_mlp_classification)

# Imputar valores nulos en variables numéricas
numerical_cols_mlp = ["carat", "depth", "table", "price", "x", "y", "z"]

imputer_numerical_mlp = Imputer(
    inputCols=numerical_cols_mlp,
    outputCols=[c + "_imputed" for c in numerical_cols_mlp],
    strategy="median"  # Se reemplazan nulos con la mediana
)

# Aplicar imputación
df_mlp_classification = imputer_numerical_mlp.fit(df_mlp_classification).transform(df_mlp_classification)

# Mostrar los primeros registros después de la imputación
df_mlp_classification.select("color_indexed", "color_imputed", "clarity_indexed", "clarity_imputed").show(10)
df_mlp_classification.select("carat", "carat_imputed", "depth", "depth_imputed").show(10)

+-------------+-------------+---------------+---------------+
|color_indexed|color_imputed|clarity_indexed|clarity_imputed|
+-------------+-------------+---------------+---------------+
|          1.0|          1.0|            2.0|            2.0|
|          1.0|          1.0|            0.0|            0.0|
|          1.0|          1.0|            3.0|            3.0|
|          5.0|          5.0|            1.0|            1.0|
|          6.0|          6.0|            2.0|            2.0|
|          6.0|          6.0|            4.0|            4.0|
|          5.0|          5.0|            5.0|            5.0|
|          3.0|          3.0|            0.0|            0.0|
|          1.0|          1.0|            1.0|            1.0|
|          3.0|          3.0|            3.0|            3.0|
+-------------+-------------+---------------+---------------+
only showing top 10 rows

+-----+-------------+-----+-------------+
|carat|carat_imputed|depth|depth_imputed|
+-----+-------------+-

In [84]:
# Aplicar One-Hot Encoding a las variables categóricas imputadas
encoders_onehot_mlp = [
    OneHotEncoder(inputCol="color_imputed", outputCol="color_onehot"),
    OneHotEncoder(inputCol="clarity_imputed", outputCol="clarity_onehot")
]

# Aplicar transformación
for encoder in encoders_onehot_mlp:
    df_mlp_classification = encoder.fit(df_mlp_classification).transform(df_mlp_classification)

# Verificar que las columnas codificadas han sido creadas correctamente
df_mlp_classification.select("color_imputed", "color_onehot", "clarity_imputed", "clarity_onehot").show(10, truncate=False)

+-------------+-------------+---------------+--------------+
|color_imputed|color_onehot |clarity_imputed|clarity_onehot|
+-------------+-------------+---------------+--------------+
|1.0          |(6,[1],[1.0])|2.0            |(7,[2],[1.0]) |
|1.0          |(6,[1],[1.0])|0.0            |(7,[0],[1.0]) |
|1.0          |(6,[1],[1.0])|3.0            |(7,[3],[1.0]) |
|5.0          |(6,[5],[1.0])|1.0            |(7,[1],[1.0]) |
|6.0          |(6,[],[])    |2.0            |(7,[2],[1.0]) |
|6.0          |(6,[],[])    |4.0            |(7,[4],[1.0]) |
|5.0          |(6,[5],[1.0])|5.0            |(7,[5],[1.0]) |
|3.0          |(6,[3],[1.0])|0.0            |(7,[0],[1.0]) |
|1.0          |(6,[1],[1.0])|1.0            |(7,[1],[1.0]) |
|3.0          |(6,[3],[1.0])|3.0            |(7,[3],[1.0]) |
+-------------+-------------+---------------+--------------+
only showing top 10 rows



In [85]:
# Ensamblar todas las características en una sola columna "features"
all_features_mlp = ["carat_imputed", "depth_imputed", "table_imputed", "price_imputed", 
                    "x_imputed", "y_imputed", "z_imputed", "color_onehot", "clarity_onehot"]

assembler_all_mlp = VectorAssembler(inputCols=all_features_mlp, outputCol="features")

# Aplicar ensamblado
df_mlp_classification = assembler_all_mlp.transform(df_mlp_classification)

# Mostrar la nueva columna "features"
df_mlp_classification.select("features").show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|(20,[0,1,2,3,4,5,6,8,15],[0.23000000417232513,61.5,55.0,326.0,3.950000047683716,3.9800000190734863,2.430000066757202,1.0,1.0])              |
|(20,[0,1,2,3,4,5,6,8,13],[0.20999999344348907,59.79999923706055,61.0,326.0,3.890000104904175,3.8399999141693115,2.309999942779541,1.0,1.0]) |
|(20,[0,1,2,3,4,5,6,8,16],[0.23000000417232513,56.900001525878906,65.0,327.0,4.050000190734863,4.070000171661377,2.309999942779541,1.0,1.0]) |
|(20,[0,1,2,3,4,5,6,12,14],[0.28999999165534973,62.400001525878906,58.0,334.0,4.199999809265137,4.230000019073486,2.630000114440918,1.0,1.0])|

In [86]:
# Normalizar la columna "features"
scaler_mlp = MinMaxScaler(inputCol="features", outputCol="features_scaled")

# Aplicar escalado
scaler_model = scaler_mlp.fit(df_mlp_classification)
df_mlp_classification = scaler_model.transform(df_mlp_classification)

# Mostrar la columna "features_scaled"
df_mlp_classification.select("features_scaled").show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features_scaled                                                                                                                                                                 |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(20,[0,1,2,4,5,6,8,15],[0.006237006191921662,0.5138888888888888,0.23076923076923078,0.3677839973801482,0.06757215477023024,0.076415098272242,1.0,1.0])                          |
|(20,[0,1,2,4,5,6,8,13],[0.0020789999986709777,0.4666666454739041,0.34615384615384615,0.3621974104101101,0.06519524303377361,0.07264150937737782,1.0,1.0])                       |
|(20,[0,1,2,3,4,5,6,8,16],[0.006237006191921662,0.3861111534966363,0.42307692307692313,5.406282099799968E

In [87]:
# Determinar el número de características y clases
num_features = df_mlp_classification.select("features_scaled").first()[0].size
num_classes = df_mlp_classification.select("label").distinct().count()

# Definir la estructura de la red neuronal
layers = [num_features, 64, 32, num_classes]

# Imprimir detalles de la red neuronal
print(f"Número de características de entrada: {num_features}")
print(f"Número de clases de salida: {num_classes}")
print(f"Estructura de la red neuronal (capas): {layers}")

Número de características de entrada: 20
Número de clases de salida: 5
Estructura de la red neuronal (capas): [20, 64, 32, 5]


In [88]:
# Definir el modelo MLP
mlp_classifier = MultilayerPerceptronClassifier(
    featuresCol="features_scaled",  # Usamos la versión escalada de las features
    labelCol="label",
    layers=layers,
    seed=42,
    maxIter=100
)

# Crear pipeline con todas las etapas
pipeline_mlp = Pipeline(stages=[mlp_classifier])

# Dividir datos en entrenamiento (80%) y prueba (20%)
df_train_mlp, df_test_mlp = df_mlp_classification.randomSplit([0.8, 0.2], seed=42)

# Entrenar el modelo
pipeline_model_mlp = pipeline_mlp.fit(df_train_mlp)

# Hacer predicciones en el conjunto de prueba
df_pred_mlp = pipeline_model_mlp.transform(df_test_mlp)

# Mostrar algunas predicciones
df_pred_mlp.select("label", "prediction").show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       2.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       2.0|
|  3.0|       4.0|
|  3.0|       3.0|
|  3.0|       2.0|
|  3.0|       0.0|
+-----+----------+
only showing top 10 rows



In [89]:
# Definir evaluadores
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Evaluar el modelo MLP
accuracy_mlp = evaluator_accuracy.evaluate(df_pred_mlp)
f1_mlp = evaluator_f1.evaluate(df_pred_mlp)
precision_mlp = evaluator_precision.evaluate(df_pred_mlp)
recall_mlp = evaluator_recall.evaluate(df_pred_mlp)

# Mostrar resultados
print("\n**Evaluación del Modelo MLP**")
print(f"Accuracy: {accuracy_mlp:.4f}")
print(f"F1-Score: {f1_mlp:.4f}")
print(f"Precision: {precision_mlp:.4f}")
print(f"Recall: {recall_mlp:.4f}")


**Evaluación del Modelo MLP**
Accuracy: 0.5653
F1-Score: 0.5060
Precision: 0.5103
Recall: 0.5653


In [90]:
# Definir la cuadrícula de hiperparámetros para MLP
paramGrid_mlp = (
    ParamGridBuilder()
    .addGrid(mlp_classifier.layers, [
        [num_features, 32, num_classes],      # Estructura más simple
        [num_features, 64, 32, num_classes],  # Estructura actual (baseline)
        [num_features, 128, 64, 32, num_classes]  # Red más profunda
    ])
    .addGrid(mlp_classifier.maxIter, [100, 200])  # Número de iteraciones
    .build()
)

# Configurar CrossValidator
crossval_mlp = CrossValidator(
    estimator=pipeline_mlp,               # Pipeline de MLP
    estimatorParamMaps=paramGrid_mlp,      # Hiperparámetros
    evaluator=evaluator_f1,                # Evaluar con F1-Score
    numFolds=3,                            # 3-Fold Cross Validation
    parallelism=4,                          # Procesamiento en paralelo
    seed=42
)

# Entrenar el modelo optimizado
cv_model_mlp = crossval_mlp.fit(df_train_mlp)

# Hacer predicciones con el mejor modelo encontrado
df_pred_cv_mlp = cv_model_mlp.transform(df_test_mlp)

# Evaluar el modelo optimizado
accuracy_cv_mlp = evaluator_accuracy.evaluate(df_pred_cv_mlp)
f1_cv_mlp = evaluator_f1.evaluate(df_pred_cv_mlp)
precision_cv_mlp = evaluator_precision.evaluate(df_pred_cv_mlp)
recall_cv_mlp = evaluator_recall.evaluate(df_pred_cv_mlp)

# Mostrar los resultados
print("\n**Evaluación del Mejor Modelo MLP (GridSearch + CrossValidation)**")
print(f"Accuracy: {accuracy_cv_mlp:.4f}")
print(f"F1-Score: {f1_cv_mlp:.4f}")
print(f"Precision: {precision_cv_mlp:.4f}")
print(f"Recall: {recall_cv_mlp:.4f}")


**Evaluación del Mejor Modelo MLP (GridSearch + CrossValidation)**
Accuracy: 0.6713
F1-Score: 0.6415
Precision: 0.6457
Recall: 0.6713


In [91]:
# Guardar el mejor modelo optimizado
cv_model_mlp.bestModel.write().overwrite().save("models/best_mlp_classification_optimized")

print("\nMejor modelo MLP optimizado guardado con éxito.")


Mejor modelo MLP optimizado guardado con éxito.


## **Análisis Final del Modelo MLP Optimizado**  

Después de aplicar **GridSearch + CrossValidation**, se obtuvo el mejor modelo de red neuronal con los siguientes resultados:  

### **Resultados de Evaluación**  
| Métrica    | Valor  |
|------------|--------|
| **Accuracy**  | `0.6713` |
| **F1-Score**  | `0.6415` |
| **Precision** | `0.6457` |
| **Recall**    | `0.6713` |

---

### **Interpretación de los Resultados**  

1️ **Mejora en el Rendimiento**  
- Comparado con el modelo base, que tenía un **Accuracy de 0.5653**, la versión optimizada logra un **+10% de mejora en precisión general**.  
- El **F1-Score** también mejora significativamente, indicando un mejor balance entre **precisión** y **recall**.  

2️ **Generalización y Robustez**  
- Gracias a la **validación cruzada**, el modelo evita **overfitting** y generaliza mejor a nuevos datos.  
- La precisión aumentada sugiere que el modelo **distingue mejor las categorías de `cut`**, aunque aún hay margen de mejora.  

3️ **Posibles Mejoras Futuras**  
- **Más capas y neuronas**: Explorar redes más profundas con estructuras como `[num_features, 256, 128, 64, 32, num_classes]`.  
- **Más datos de entrenamiento**: Aumentar la cantidad de datos puede mejorar la capacidad de aprendizaje del modelo.  
- **Hiperparámetros adicionales**: Ajustar el **learning rate**, la **función de activación**, y el **batch size**.  

---

### **Conclusión**  
**El modelo optimizado con MLP y GridSearch mejora notablemente el rendimiento respecto a la versión inicial.**  
**Se lograron predicciones más precisas y balanceadas entre clases.**  
**Aún hay oportunidades de optimización explorando redes más profundas y ajustes más finos en los hiperparámetros.**  

---

### **Guardado del Mejor Modelo**  
El modelo optimizado ha sido almacenado en `models/best_mlp_classification` para su uso en futuras predicciones sin necesidad de volver a entrenarlo.  

```python
cv_model_mlp.bestModel.write().overwrite().save("models/best_mlp_classification")
print("\nMejor modelo MLP guardado con éxito.")